In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

from setup_imports import *  # noqa: F401,F403


from src.story import get_story, get_all_stories
from src.challenges.models import ChallengeRecord, get_challenge
from langcodes import Language

COLLECTION = "LM1000"
DECK = "Pack05"

In [9]:
target_lang = Language.get("sv-SE")
print(target_lang.display_name())

Swedish (Sweden)


In [ ]:
stories = get_all_stories(collection=COLLECTION)

In [7]:
story = get_story(story_title_hash="the_winter_restaurant_challenge_c5056c")

In [ ]:
challenge = get_challenge("a_night_at_the_restaurant_13f019")

In [ ]:
challenge.publish(Language.get("es-ES"))

In [ ]:
for s in stories:
    print(s.deck)

In [8]:
challenge = ChallengeRecord.create(story)
challenge.upload()

2026-02-15 19:10:06 - audio-language-trainer - INFO - models.py:202 - Generating challenges for story 'The Winter Restaurant Challenge'
2026-02-15 19:10:21 - audio-language-trainer - INFO - models.py:211 - Created ChallengeRecord with 5 scenarios for story 'The Winter Restaurant Challenge'
2026-02-15 19:10:21 - audio-language-trainer - INFO - models.py:295 - Uploaded ChallengeRecord to Firestore: challenges/the_winter_restaurant_challenge_c5056c


In [10]:
challenge.publish(target_language=target_lang)

2026-02-15 19:10:27 - audio-language-trainer - INFO - models.py:260 - Published challenge at gs://audio-language-trainer-stories/challenges/en-GB/sv-SE/the_winter_restaurant_challenge_c5056c/index.html
2026-02-15 19:10:27 - audio-language-trainer - INFO - models.py:274 - Finished publishing challenge to GCS at challenges/en-GB/sv-SE/the_winter_restaurant_challenge_c5056c/ for en-GB -> sv-SE
2026-02-15 19:10:27 - audio-language-trainer - INFO - models.py:295 - Uploaded ChallengeRecord to Firestore: challenges/the_winter_restaurant_challenge_c5056c


'https://storage.googleapis.com/audio-language-trainer-stories/challenges/en-GB/sv-SE/the_winter_restaurant_challenge_c5056c/index.html'